# Data camp 실습 코드

## 1. Supervised Learning with scikit-learn

### (1) Classification

In [4]:
import warnings; warnings.filterwarnings('ignore')
import sklearn
sklearn.__version__

# sklearn.neighbors.classification was renamed to sklearn.neighbors._classification in version 0.22.X
# Downgrade to scikit-learn version <= 0.21.3 to fix this problem
# (https://github.com/ageitgey/face_recognition/issues/1262)

# pip install --user --upgrade scikit-learn==0.21.3

'0.21.3'

#### **Classifying labels of unseen data**
1. Build a model
2. Model learns from the labeled data we pass to it (Labeled data = training data)
3. pass unlabeled data to the model as input
4. Model predicts the labels of the unseen data

#### k-Nearest Neighbors
* Predict the label of a data point by
  * Looking at the **k** closest labeled data points
  * Taking a majority vote

In [6]:
# KNN 사용법
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

X = churn_df[['total_day_charge', 'total_eve_charge']].values
y = churn_df['churn'].values
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X, y)

X_new = np.array([[56.8, 17.5],
                 [24.4,24.1],
                 [50.1, 10.9]])
print(X_new.shape)

predictions = knn.predict(X_new)
print(f'Predictions: {predictions}')

#### Measuring model performance

* How do we measure accuracy?
* Could compute accuracy on the data used to fit the classifier
* Not indicative of ability to generalize
* Split data -> Training set / Test set

In [ ]:
# Train/test split
from sklearn.medel_selection import train_test_split
X = churn_df.drop("churn", axis=1).values
y = churn_df["churn"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21, stratify=y)

knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

* Model Complexity
  * Larger k = less complex model = can cause underfitting
  * Smaller k = more complex model = can lead to overfitting

In [ ]:
# Model complexity and over/underfitting
import matplotlib.pyplot as plt

train_accuracies = {}
test_accuracies = {}
neighbors = np.arange(1, 26)

for neighbor in neighbors:
    knn = KNeighborsClassifier(n_neighbors=neighbor)
    knn.fit(X_train, y_train)
    train_accuracies[neighbor] = knn.score(X_train, y_train)
    test_accuracies[neighbor] = knn.score(X_test, y_test)
    
print(neighbors, '\n', train_accuracies, '\n', test_accuracies)
    
plt.figure(figsize=(8, 6))
plt.title("KNN: Varying Number of Neighbors")
plt.plot(neighbors, train_accuracies.values(), label="Training Accuracy")
plt.plot(neighbors, test_accuracies.values(), label="Testing Accuracy")
plt.xlabel("Number of Neighbors")
plt.ylabel("Accuracy")

### (2) Regression

#### Introduction to regression

In [ ]:
# Creating features

import numpy as np

# Create X from the radio column's values
X = sales_df['radio'].values

# Create y from the sales column's values
y = sales_df['sales'].values

# Reshape X
X = X.reshape(-1, 1)

# Check the shape of the features and targets
print(X.shape, y.shape)

In [ ]:
# Building a linear regression model

# Import LinearRegression
from sklearn.linear_model import LinearRegression

# Create the model
reg = LinearRegression()

# Fit the model to the data
reg.fit(X, y)

# Make predictions
predictions = reg.predict(X)

print(predictions[:5])

In [ ]:
# Visualizing a linear regression model

# Import matplotlib.pyplot
import matplotlib.pyplot as plt

# Create scatter plot
plt.scatter(X, y, color="blue")

# Create line plot
plt.plot(X, predictions, color="red")
plt.xlabel("Radio Expenditure ($)")
plt.ylabel("Sales ($)")

# Display the plot
plt.show()

In [ ]:
# Fit and predict for regression
# Create X and y arrays
X = sales_df.drop("sales", axis=1).values
y = sales_df["sales"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Instantiate the model
reg = LinearRegression()

# Fit the model to the data
reg.fit(X_train, y_train)

# Make predictions
y_pred = reg.predict(X_test)
print("Predictions: {}, Actual Values: {}".format(y_pred[:2], y_test[:2]))

In [ ]:
# Regression preformance
# Import mean_squared_error
from sklearn.metrics import mean_squared_error

# Compute R-squared
r_squared = reg.score(X_test, y_test)

# Compute RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Print the metrics
print("R^2: {}".format(r_squared))
print("RMSE: {}".format(rmse))

#### Cross-validation

In [ ]:
# Cross-validation for R-squared

# Import the necessary modules
from sklearn.model_selection import cross_val_score, KFold

# Create a KFold object
kf = KFold(n_splits=6, shuffle=True, random_state=5)

reg = LinearRegression()

# Compute 6-fold cross-validation scores
cv_scores = cross_val_score(reg, X, y, cv=kf)

# Print scores
print(cv_scores)

In [ ]:
# Analyzing cross-validation metrics

# Print the mean
print(np.mean(cv_results))

# Print the standard deviation
print(np.std(cv_results))

# Print the 95% confidence interval
print(np.quantile(cv_results, [0.025, 0.975]))

#### Regularized regression

* Ridge
  * Ridge regression performs regularization by computing the squared values of the model parameters multiplied by alpha and adding them to the loss function.

In [ ]:
# Regularized regression: Ridge

# Import Ridge
from sklearn.linear_model import Ridge
alphas = [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]
ridge_scores = []
for alpha in alphas:
  
  # Create a Ridge regression model
    ridge = Ridge(alpha=alpha)
  
  # Fit the data
    ridge.fit(X_train, y_train)
  
  # Obtain R-squared
    score = ridge.score(X_test, y_test)
    ridge_scores.append(score)
print(ridge_scores)

* Lasso
  * Lasso can select important features of a dataset (because shrinks the coefficients of less important features to zero)

In [ ]:
# Lasso regression for feature importance

# Import Lasso
from sklearn.linear_model import Lasso

# Instantiate a lasso regression model
lasso = Lasso(alpha=0.3)

# Fit the model to the data
lasso.fit(X, y)

# Compute and print the coefficients
lasso_coef = lasso.fit(X, y).coef_
print(lasso_coef)
plt.bar(sales_columns, lasso_coef)
plt.xticks(rotation=45)
plt.show()

### (3) Fine-Tuing Your Model

#### Assessing a diabetes prediction classifier

In [ ]:
# Import confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

knn = KNeighborsClassifier(n_neighbors=6)

# Fit the model to the training data
knn.fit(X_train, y_train)

# Predict the labels of the test data: y_pred
y_pred = knn.predict(X_test)

# Generate the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

#### Building a logistic regression model

In [ ]:
# Import LogisticRegression
from sklearn.linear_model import LogisticRegression

# Instantiate the model
logreg = LogisticRegression()

# Fit the model
logreg.fit(X_train, y_train)

# Predict probabilities
y_pred_probs = logreg.predict_proba(X_test)[:, 1]

print(y_pred_probs[:10])

#### The ROC curve

In [ ]:
# Import roc_curve
from sklearn.metrics import roc_curve

# Generate ROC curve values: fpr, tpr, thresholds
fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)

plt.plot([0, 1], [0, 1], 'k--')

# Plot tpr against fpr
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Diabetes Prediction')
plt.show()

#### ROC AUC

In [ ]:
# Import roc_auc_score
from sklearn.metrics import roc_auc_score

# Calculate roc_auc_score
print(roc_auc_score(y_test, y_pred_probs))

# Calculate the confusion matrix
print(confusion_matrix(y_test, y_pred))

# Calculate the classification report
print(classification_report(y_test, y_pred))

#### Hyperparameter tuning with GridSearchCV

In [ ]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

# Set up the parameter grid
param_grid = {"alpha": np.linspace(0.00001, 1, 20)}

# Instantiate lasso_cv
lasso_cv = GridSearchCV(Lasso(), param_grid, cv=kf)

# Fit to the training data
lasso_cv.fit(X_train, y_train)
print("Tuned lasso paramaters: {}".format(lasso_cv.best_params_))
print("Tuned lasso score: {}".format(lasso_cv.best_score_))

#### Hyperparameter uning with RandomizedSearchCV

In [ ]:
# Create the parameter space
params = {"penalty": ["l1", "l2"],
         "tol": np.linspace(0.0001, 1.0, 50),
         "C": np.linspace(0.1, 1.0, 50),
         "class_weight": ["balanced", {0:0.8, 1:0.2}]}

# Instantiate the RandomizedSearchCV object
logreg_cv = RandomizedSearchCV(logreg, params, cv=kf)

# Fit the data to the model
logreg_cv.fit(X_train, y_train)

# Print the tuned parameters and score
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_))
print("Tuned Logistic Regression Best Accuracy Score: {}".format(logreg_cv.best_score_))

## (4) Preprocessing data

#### Creating dummy variables

In [ ]:
# Create music_dummies
music_dummies = pd.get_dummies(music_df, drop_first=True)

# Print the new DataFrame's shape
print("Shape of music_dummies: {}".format(music_dummies.shape))

#### Regression with categorical features

In [ ]:
# Create X and y
X = music_dummies.drop(columns='popularity', axis=1).values
y = music_dummies['popularity'].values

# Instantiate a ridge model
ridge = Ridge(alpha=0.2)

# Perform cross-validation
scores = cross_val_score(ridge, X, y, cv=kf, scoring="neg_mean_squared_error")  # https://ai.stackexchange.com/questions/9022/can-the-mean-squared-error-be-negative

# Calculate RMSE
rmse = np.sqrt(-scores)
print("Average RMSE: {}".format(np.mean(rmse)))
print("Standard Deviation of the target array: {}".format(np.std(y)))


#### Droping missing data

In [ ]:
# Print missing values for each column
print(music_df.isna().sum().sort_values())

# Remove values where less than 5% are missing
music_df = music_df.dropna(subset=["genre", "popularity", "loudness", "liveness", "tempo"])

# Convert genre to a binary feature
music_df["genre"] = np.where(music_df["genre"] == "Rock", 1, 0)

print(music_df.isna().sum().sort_values())
print("Shape of the `music_df`: {}".format(music_df.shape))

#### Pipeline for song genre prediction: 1

In [ ]:
# Import modules
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Instantiate an imputer
imputer = SimpleImputer()

# Instantiate a knn model
knn = KNeighborsClassifier(n_neighbors=3)

# Build steps for the pipeline
steps = [("imputer", imputer), 
         ("knn", knn)]

#### Pipeline for song genre prediction: 2

In [ ]:
steps = [("imputer", imp_mean),
        ("knn", knn)]

# Create the pipeline
pipeline = Pipeline(steps)

# Fit the pipeline to the training data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Print the confusion matrix
print(confusion_matrix(y_test, y_pred))

#### Centering and scaling (Centering and scaling for regression)

In [ ]:
# Import StandardScaler
from sklearn.preprocessing import StandardScaler

# Create pipeline steps
steps = [("scaler", StandardScaler()),
         ("lasso", Lasso(alpha=0.5))]

# Instantiate the pipeline
pipeline = Pipeline(steps)
pipeline.fit(X_train, y_train)

# Calculate and print R-squared
print(pipeline.score(X_test, y_test))

#### Centering and scaling for classification

In [11]:
# Build the steps
steps = [("scaler", StandardScaler()),
         ("logreg", LogisticRegression())]
pipeline = Pipeline(steps)

# Create the parameter space
parameters = {"logreg__C": np.linspace(0.001, 1.0, 20)}                          # 모델의 하이퍼파라미터 name = "logreg__C"로 해야 함
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=21)

# Instantiate the grid search object
cv = GridSearchCV(pipeline, param_grid=parameters)

# Fit to the training data
cv.fit(X_train, y_train)
print(cv.best_score_, "\n", cv.best_params_)

#### Evaluating multiple models (Visualizing regression model performance)

* 상황마다 사용해야 하는 모델은 다름
* 기능이 적을수록 더 간단하고 빠름 (+ 해석 가능한 모델)

It's all in the metrics
* Regression model performance:
    * RMSE
    * R-squared
* Classification model performance:
    * Accuracy
    * Confusion matrix
    * Precision, recall, F1-score
    * ROC AUC  
  
A note on scaling
* Models affected by scaling
    * KNN
    * Linear Regression (+ Ridge, Lasso)
    * Logistic Regression
    * Artificiai Neural Network
* Best to scale our data before evaluating models

In [ ]:
models = {"Linear Regression": LinearRegression(), "Ridge": Ridge(alpha=0.1), "Lasso": Lasso(alpha=0.1)}
results = []

# Loop through the models' values
for model in models.values():
    kf = KFold(n_splits=6, random_state=42, shuffle=True)
  
    # Perform cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=kf)
  
    # Append the results
    results.append(cv_scores)

# Create a box plot of the results
plt.boxplot(results, labels=models.keys())
plt.show()

#### Predicting on the test set

In [ ]:
# Import mean_squared_error
from sklearn.metrics import mean_squared_error

for name, model in models.items():
  
    # Fit the model to the training data
    model.fit(X_train_scaled, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test_scaled)

    # Calculate the test_rmse
    test_rmse = mean_squared_error(y_test, y_pred, squared=False)
    print("{} Test Set RMSE: {}".format(name, test_rmse))

#### Visualizing classification model performance

In [ ]:
# Create models dictionary
models = {"Logistic Regression": LogisticRegression(), "KNN": KNeighborsClassifier(), "Decision Tree Classifier": DecisionTreeClassifier()}
results = []

# Loop through the models' values
for model in models.values():
  
    # Instantiate a KFold object
    kf = KFold(n_splits=6, random_state=12, shuffle=True)

    # Perform cross-validation
    cv_results = cross_val_score(model, X_train_scaled, y_train, cv=kf)
    results.append(cv_results)
plt.boxplot(results, labels=models.keys())
plt.show()

#### Pipeline for predicting song popularity

In [ ]:
# Create steps
steps = [("imp_mean", SimpleImputer()), 
         ("scaler", StandardScaler()), 
         ("logreg", LogisticRegression())]

# Set up pipeline
pipeline = Pipeline(steps)
params = {"logreg__solver": ["newton-cg", "saga", "lbfgs"],
         "logreg__C": np.linspace(0.001, 1.0, 10)}

# Create the GridSearchCV object
tuning = GridSearchCV(pipeline, param_grid=params)
tuning.fit(X_train, y_train)
y_pred = tuning.predict(X_test)

# Compute and print performance
print("Tuned Logistic Regression Parameters: {}, Accuracy: {}".format(tuning.best_params_, tuning.best_score_))

# 이론

## (3) Fine-Tuning Your Model

* 오차율, 정확도(accuracy)는 분류 분석에서 가장 자주 사용하는 2가지 성능척도. 이진 분류 뿐만 아니라 다중 분류에서도 자주 사용.  
    - 오차율 : 모든 샘플 수에서 잘못 분류한 샘플 수가 차지하는 비율
    - 정확도 : 전체 샘플 수에서 정확히 분류한 샘플 수가 차지하는 비율
* 오차율, 정확도는 자주 사용되지만 모든 문제에 활용되진 못함. 
    - ex) 수박 장수가 수박을 손수레에 가득 담아 왔다. 그는 훈련된 모델을 통해 수박들을 분류하려고 하는데 여기서 오차율은 덜 익은 수박으로 분류되는 판별 오차율을 나타낼 수 있다. 그러나, 우리가 알고자 하는 건 '골라낸 수박 중에 잘 익은 수박의 비율', 혹은 '모든 잘 익은 수박 중 선택될 비율'이다. 여기서 오차율은 도움을 줄 수 없다.
    - ex) 정보 검색 중 일반적으로 확인하고자 하는 건 '검색된 자료 중 내가 관심있어 할 내용의 비율'이다.  

* 정밀도(precision)와 재현율(recall)이 이러한 요구에 맞는 성능 척도.  
    - 이진 분류 문제에서 실제 클래스와 학습기가 예측 분류한 클래스의 조합은 실제양성(true positive), 거짓양성(false positive), 실제음성(true negative), 거짓음성(false negative) 총 4가지로 요약
    - 각각 TP, FP, TN, FN으로 나타내며 TP + FP + TN + FN = 총 샘플 수'이다.
    - 이러한 분류 결과를 혼동행렬(confusion matrix)라고 함
        - 정밀도: 양성 예측의 정확도. TP / TP + FP
        - 재현율: 분류기가 정확하게 감지한 양성 샘플의 비율 (민감도 sensitivity, 진짜 양성 비율 true positive rate(TPR) 이라고도 함). TP / TP + FN
* F1-Score
    - 정밀도와 재현율을 하나의 숫자로 만든 것
    - 정밀도와 재현율의 조화평균
    - F1 = 2 / (1/정밀도 + 1/재현율) = 2 * (정밀도 * 재현율 / 정밀도 + 재현율) = TP / (TP + (FN + FP) / 2)

* Logistic regresson 을 활용한 평가
* ROC Curve (receiver operating characterstic Curve)
    * 이진 분류에서 많이 사용
    * 정밀도에 대한 재현율 곡선이 아님. 거짓 양성 비율(False positive rate, FPR)에 대한 진짜 양성 비율(True positive rate, TPR)의 곡선
        * FPR = 양성으로 잘못 분류된 음성 샘풀의 비율. 1 - TNR (음성으로 정확하게 분류한 음성 샘플의 비율)
            * TNR = 특이도(speicificity)라고도 함
    * 즉, ROC 곡선은 민감도(재현율)에 대한 1-특이도 그래프
    * ROC 곡선이 좌측 하단과 우측 상단을 이은 직선에 가까울수록 성능이 떨어지는 것이며, 멀어질수록 성능이 뛰어난 것
* ROC AUC
    * 곡선 아래의 면적(area under the curve)을 측정하면 분류기들을 비교할 수 있음
    * 완벽한 분류기는 ROC의 AUC가 1, 완전한 랜덤 분류기는 0.5

# 실습 코드